# Import Necessary Headers

In [1]:
import pandas as pd
import numpy as np
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Load Updated CSV into Pandas Dataframe¶

In [2]:
# Hong et al. Dataset
df = pd.read_csv("hong_et_al/df_updt.csv")

In [3]:
# Drop unnecessary columns
df = df.drop(columns=df.filter(like='chrkidneydisease').columns)
df = df.drop(columns=df.filter(like='chrkidneydisease').columns)
df = df.drop(columns=df.filter(like='(pcp)screen').columns)
df = df.drop(columns=df.filter(regex='^cc_').columns)
df = df.dropna(subset=['egfr_CKD_EPI'])

# Impute median data for any remaining NaN values in numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Impute mode data for any remaining NaN values in categorical columns
categorical_cols = ['dep_name', 'gender', 'ethnicity', 'race', 'lang', 'religion',
                    'maritalstatus', 'employstatus', 'insurance_status', 'disposition', 'arrivalmode',
                    'arrivalmonth', 'arrivalday', 'arrivalhour_bin', 'previousdispo']

# One-hot encode categorical features
df = pd.get_dummies(df, columns=categorical_cols)

# Ensure no infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.median(), inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212834 entries, 6 to 560485
Columns: 853 entries, esi to previousdispo_Transfer to Another Facility
dtypes: bool(100), float64(386), int64(367)
memory usage: 1.2 GB


# Define Train and Test Data Splits

In [5]:
# Exclude target variable from features frame
X = df.drop(columns=df.filter(like='egfr').columns)
y = df['egfr_CKD_EPI']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Ensure no NaN values in tensors
assert not X_train_tensor.isnan().any(), "NaN values found in X_train_tensor"
assert not y_train_tensor.isnan().any(), "NaN values found in y_train_tensor"

# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define ResNet NN

In [6]:
# Define single residual block for tabular data
class ResNetBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResNetBlock, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        residual = x # Save the input for the residual connection
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = out + residual # Add the residual (input) to the output
        out = self.relu(out)
        return out

# Define ResNet model for tabular data
class ResNetForTabular(nn.Module):
    def __init__(self, input_dim, num_blocks, hidden_dim):
        super(ResNetForTabular, self).__init__()
        self.initial_fc = nn.Linear(input_dim, hidden_dim) # Initial fully connected layer
        self.res_blocks = nn.Sequential(
            *[ResNetBlock(hidden_dim, hidden_dim) for _ in range(num_blocks)] # Stack multiple residual blocks
        )
        self.final_fc = nn.Linear(hidden_dim, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.initial_fc(x)
        out = self.relu(out)
        out = self.res_blocks(out)
        out = self.final_fc(out)
        return out

# Hyperparameters
input_dim = X_train.shape[1]
num_blocks = 3
hidden_dim = 64


# Initialize the model
model = ResNetForTabular(input_dim, num_blocks, hidden_dim)

# Load model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNetForTabular(
  (initial_fc): Linear(in_features=840, out_features=64, bias=True)
  (res_blocks): Sequential(
    (0): ResNetBlock(
      (fc1): Linear(in_features=64, out_features=64, bias=True)
      (relu): ReLU()
      (fc2): Linear(in_features=64, out_features=64, bias=True)
    )
    (1): ResNetBlock(
      (fc1): Linear(in_features=64, out_features=64, bias=True)
      (relu): ReLU()
      (fc2): Linear(in_features=64, out_features=64, bias=True)
    )
    (2): ResNetBlock(
      (fc1): Linear(in_features=64, out_features=64, bias=True)
      (relu): ReLU()
      (fc2): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (final_fc): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

# Training Loop

In [7]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(test_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch 1/15, Training Loss: 345.94474798429735, Validation Loss: 77.56979125541251
Epoch 2/15, Training Loss: 63.412447955544756, Validation Loss: 37.70549375087291
Epoch 3/15, Training Loss: 35.26972695429134, Validation Loss: 29.99616678054626
Epoch 4/15, Training Loss: 27.418374719587497, Validation Loss: 21.659078857920193
Epoch 5/15, Training Loss: 23.530242910424555, Validation Loss: 20.505838261710274
Epoch 6/15, Training Loss: 15.473468647734288, Validation Loss: 22.28317565030164
Epoch 7/15, Training Loss: 12.547863411034287, Validation Loss: 12.517916227604175
Epoch 8/15, Training Loss: 9.67694901470312, Validation Loss: 11.340962945877969
Epoch 9/15, Training Loss: 8.255225988135038, Validation Loss: 13.473149044019681
Epoch 10/15, Training Loss: 7.049916871191639, Validation Loss: 9.717038580963203
Epoch 11/15, Training Loss: 5.8056324273291855, Validation Loss: 6.401598432400563
Epoch 12/15, Training Loss: 5.434525909212378, Validation Loss: 5.814947381033912
Epoch 13/15, T

# Evaluate Accuracy on Test Set

In [8]:
# Switch the model to evaluation mode
model.eval()

# Initialize lists to store predictions and true values
all_predictions = []
all_labels = []

# Predict and evaluate
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Concatenate all batches into a single array
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Calculate metrics
mse = mean_squared_error(all_labels, all_predictions)
mae = mean_absolute_error(all_labels, all_predictions)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 7.381031513214111
Mean Absolute Error: 1.3308500051498413


# SHAP Explainer

In [16]:
# Select a subset of the training data to use as the background dataset for SHAP
background = X_train_tensor[:100].to(device)

# Initialize the SHAP DeepExplainer with the model and background dataset
explainer = shap.DeepExplainer(model, background)

# Define the list to store SHAP values
shap_list = []

# Set the batch size for SHAP computation
batch_size = 256

# Convert the tensor to numpy for slicing
X_test_np = X_test_tensor.cpu().numpy()

# Create indices for batching
idxs = list(map(int, np.linspace(0, X_test_np.shape[0] - 1, int(X_test_np.shape[0] / batch_size))))

# Use tqdm to display a progress bar for SHAP value computation
for i1, i2 in tqdm(zip(idxs[:-1], idxs[1:]), total=len(idxs) - 1):
    # Compute SHAP values for the current batch
    shaps = explainer.shap_values(torch.from_numpy(X_test_np[i1:i2, :]).to(device), check_additivity=False)
    
    # If the model has a single output, shaps will be a single array
    if isinstance(shaps, list):
        shap_list.append(shaps[0])
    else:
        shap_list.append(shaps)

# Concatenate the list of SHAP values into a single array
shap_values = np.concatenate(shap_list, axis=0)

# Ensure shap_values is in the correct shape (n_samples, n_features)
if len(shap_values.shape) == 3:
    shap_values = shap_values[:, 0, :]

# Convert SHAP values to numpy arrays and move the test data back to the CPU for plotting
X_test_np = X_test_tensor.cpu().numpy()

100%|██████████████████████████████████████████████████████████████████████████████████| 165/165 [12:04<00:00,  4.39s/it]


In [17]:
# Summary plot of SHAP values
shap.summary_plot(shap_values, X_test_np, feature_names=X.columns.tolist())

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.